# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


## 1. Spark SQL Solution

In [1]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-10-13 15:04:15,278 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [2]:
! hadoop fs -copyFromLocal clickstream.csv

copyFromLocal: `clickstream.csv': File exists


In [3]:
! hadoop fs -ls

Found 2 items
drwxr-xr-x   - jovyan supergroup          0 2024-10-13 15:04 .sparkStaging
-rw-r--r--   1 jovyan supergroup   32241574 2024-10-06 12:03 clickstream.csv


In [4]:
df = se.read.csv('clickstream.csv', header=True, sep='\t')
df.show()

+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1695584127|
|    562|       507|       event|      main|1695584134|
|    562|       507|       event|      main|1695584144|
|    562|       507|       event|      main|1695584147|
|    562|       507|wNaxLlerrorU|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584160|
|    562|       507|        page|    rabota|1695584166|
|    562|       507|       event|    rabota|1695584174|
|    562|       507|       event|    rabota|1695584181|
|    562|       507|       event|    rabota|1695584189|
|    562|       507|        page|      main|1695584194|
|    562|       507|       event|      main|1695584204|
|    562|       507|       event|      main|1695

In [5]:
df.registerTempTable("df")

In [16]:
most_frequent_routes_with_cte = se.sql('''
with session_limit as (
    select user_id, session_id, min(timestamp) as first_error_timestamp
    from df
    where event_type LIKE '%error%'
    group by user_id, session_id
),

ordered_events as (
    select d.user_id, d.session_id, d.event_page, d.timestamp,
           lag(d.event_page) over (partition by d.user_id, d.session_id order by d.timestamp) as prev_page
    from df d
    left join session_limit sl on sl.user_id = d.user_id and sl.session_id = d.session_id
    where d.event_type = 'page'
      and (sl.first_error_timestamp is null or d.timestamp < sl.first_error_timestamp)
),

filtered_events as (
    select user_id, session_id, event_page, timestamp
    from ordered_events
    where event_page != prev_page or prev_page is null
),

all_routes as (
    select user_id, session_id, concat_ws('-', collect_list(event_page)) as route
    from (
        select user_id, session_id, event_page, timestamp
        from filtered_events
        order by user_id, session_id, timestamp
    )
    group by user_id, session_id
)

select route, count(distinct user_id, session_id) as cnt
from all_routes
group by route
order by cnt desc, route
limit 30
''')

most_frequent_routes_spark_sql = most_frequent_routes_with_cte.collect()

for row in most_frequent_routes_spark_sql:
    print(f"{row['route']}\t{row['cnt']}")

main	8184
main-archive	1113
main-rabota	1047
main-internet	897
main-bonus	870
main-news	769
main-tariffs	677
main-online	587
main-vklad	518
main-rabota-archive	170
main-archive-rabota	167
main-bonus-archive	143
main-rabota-bonus	139
main-bonus-rabota	135
main-news-rabota	135
main-archive-internet	132
main-rabota-news	130
main-internet-rabota	129
main-archive-news	126
main-rabota-internet	124
main-internet-archive	123
main-archive-bonus	117
main-internet-bonus	115
main-tariffs-internet	114
main-news-archive	113
main-news-internet	109
main-archive-tariffs	104
main-internet-news	103
main-tariffs-archive	103
main-rabota-main	94


* **Let's include a logic explanation here:**

1. CTE **session_limit**: 
- Finds the first error for each user session. 
- For each *(user_id, session_id)* pair, the query calculates the minimum *timestamp*, which represents the first occurence of an error in that session.
- This CTE helps us determine when to stop considering page visits for the session.

2. CTE **ordered_events**: 
- Captures valid 'page' events for each session and introduces a *lag()* to track the previous page within the same session. 
- We join the *df* DataFrame with the *session_limit* to include the first error *timestamp* for each session. 
- We then filter the data to include only rows where *event_type* = 'page'. 
- The query further filters events to include those that happened before the first error. 
- The *lag()* function creates a new column *prev_page* that contains the previous page visited in the session. 
- It uses a window function *(PARTITION BY user_id, session_id ORDER BY timestamp)* to ensure the correct ordering of events within each session.

3. CTE **filtered_events**:
- Removes consecutive duplicate *event_page* entries.
- For each session: if the *event_page* is different from the *prev_page*, it's kept in the result. If *prev_page* is *NULL* (which happens for the first event is a session), that event is also kept.
- This ensures that only unique page transitions are included in the route, avoiding consecutive duplicates.

4. CTE **all_routes**:
- Creates a "route" for each session by concatenating the pages visited in the correct order.
- First, the query orders the filtered events by *user_id, session_id* and *timestamp* to ensure that the pages are in the correct sequence.
- Then, it uses *collect_list(event_page)* to gather all *event_page* values for each *(user_id, session_id)* pair and concatenate them into a single string using *concat_ws('-')*, which creates the route.
- This CTE produces the user route, which is a sequence of pages visited during each session.

5. Final query: **Most Frequent Routes**:
- Counts the number of distinct *(user_id, session_id)* pairs (i.e., unique sessions) for each route and returns the top 30 most frequent routes.
- *count(distinct user_id, session_id)* counts how many unique sessions had each specific route.
- The query then groups the results by *route* and orders by the count in descending order.
- *limit 30* ensures that only the top 30 routes are returned.

* **Let's also include here a version without cte, just for fun:**

In [7]:
most_frequent_routes_without_cte = se.sql('''
select route, count(distinct user_id, session_id) as cnt
from (
    select user_id, session_id, concat_ws("-", collect_list(event_page)) as route
    from (
        select user_id, session_id, event_page, timestamp
        from (
            select d.user_id, d.session_id, d.event_page, d.timestamp,
                   lag(d.event_page) over (partition by d.user_id, d.session_id order by d.timestamp) as prev_page
            from df d
            left join (
                select user_id, session_id, min(timestamp) as first_error_timestamp
                from df
                where event_type like '%error%'
                group by user_id, session_id
            ) sl on sl.user_id = d.user_id and sl.session_id = d.session_id
            where d.event_type = 'page' 
              and (sl.first_error_timestamp is null or d.timestamp < sl.first_error_timestamp)
            order by d.user_id, d.session_id, d.timestamp
        ) 
        where event_page != prev_page or prev_page is null
    )
    group by user_id, session_id
) 
group by route
order by cnt desc, route
limit 30
''')

most_frequent_routes_spark_sql_without_cte = most_frequent_routes_without_cte.collect()

for row in most_frequent_routes_spark_sql_without_cte:
    print(f"{row['route']}\t{row['cnt']}")

main	8184
main-archive	1113
main-rabota	1047
main-internet	897
main-bonus	870
main-news	769
main-tariffs	677
main-online	587
main-vklad	518
main-rabota-archive	170
main-archive-rabota	167
main-bonus-archive	143
main-rabota-bonus	139
main-bonus-rabota	135
main-news-rabota	135
main-archive-internet	132
main-rabota-news	130
main-internet-rabota	129
main-archive-news	126
main-rabota-internet	124
main-internet-archive	123
main-archive-bonus	117
main-internet-bonus	115
main-tariffs-internet	114
main-news-archive	113
main-news-internet	109
main-archive-tariffs	104
main-internet-news	103
main-tariffs-archive	103
main-rabota-main	94


* **Now, we should create Spark RDD Solution and Spak DF solution using the same logic and compare the results:**

## 2. Spark RDD Solution

In [8]:
df_rdd = df.rdd

In [9]:
session_limit_rdd = (df_rdd
    .filter(lambda row: 'error' in row['event_type'])
    .map(lambda row: ((row['user_id'], row['session_id']), row['timestamp']))
    .reduceByKey(min)
)

valid_events_rdd = (df_rdd
    .map(lambda row: ((row['user_id'], row['session_id']), row))
    .leftOuterJoin(session_limit_rdd)
    .filter(lambda x: x[1][0]['event_type'] == 'page' and 
                      (x[1][1] is None or x[1][0]['timestamp'] < x[1][1]))
)

def remove_consecutive_duplicates(session):
    ordered_session = sorted(session, key=lambda x: x[1])
    result = []
    prev_page = None
    for page, timestamp in ordered_session:
        if page != prev_page:
            result.append((page, timestamp))
        prev_page = page
    return result

filtered_events_rdd = (valid_events_rdd
    .map(lambda x: ((x[0][0], x[0][1]), (x[1][0]['event_page'], x[1][0]['timestamp'])))
    .groupByKey()
    .mapValues(remove_consecutive_duplicates)
)

routes_rdd = (filtered_events_rdd
    .map(lambda x: (x[0], '-'.join([page for page, _ in x[1]])))
)

most_frequent_routes_rdd = (routes_rdd
    .map(lambda x: (x[1], (x[0][0], x[0][1])))  
    .distinct()
    .map(lambda x: (x[0], 1))  
    .reduceByKey(lambda a, b: a + b)
    .sortBy(lambda x: (-x[1], x[0]))
    .take(30)
)

for route, cnt in most_frequent_routes_rdd:
    print(f"{route}\t{cnt}")

main	8184
main-archive	1113
main-rabota	1047
main-internet	897
main-bonus	870
main-news	769
main-tariffs	677
main-online	587
main-vklad	518
main-rabota-archive	170
main-archive-rabota	167
main-bonus-archive	143
main-rabota-bonus	139
main-bonus-rabota	135
main-news-rabota	135
main-archive-internet	132
main-rabota-news	130
main-internet-rabota	129
main-archive-news	126
main-rabota-internet	124
main-internet-archive	123
main-archive-bonus	117
main-internet-bonus	115
main-tariffs-internet	114
main-news-archive	113
main-news-internet	109
main-archive-tariffs	104
main-internet-news	103
main-tariffs-archive	103
main-rabota-main	94


## 3. Spark DF Solution

In [10]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

session_limit = (df
    .filter(F.col("event_type").like("%error%"))
    .groupBy("user_id", "session_id")
    .agg(F.min("timestamp").alias("first_error_timestamp"))
)

window_spec = Window.partitionBy("user_id", "session_id").orderBy("timestamp")
ordered_events = (df
    .join(session_limit, on=["user_id", "session_id"], how="left")
    .filter((F.col("event_type") == "page") & 
            ((F.col("first_error_timestamp").isNull()) | (F.col("timestamp") < F.col("first_error_timestamp"))))
    .withColumn("prev_page", F.lag("event_page").over(window_spec))
)

filtered_events = ordered_events.filter((F.col("event_page") != F.col("prev_page")) | F.col("prev_page").isNull())

filtered_events_ordered = filtered_events.orderBy("user_id", "session_id", "timestamp")

all_routes = (filtered_events_ordered
    .groupBy("user_id", "session_id")
    .agg(F.concat_ws("-", F.collect_list("event_page")).alias("route"))
)

most_frequent_routes_df = (all_routes
    .groupBy("route")
    .agg(F.countDistinct("user_id", "session_id").alias("cnt"))
    .orderBy(F.col("cnt").desc(), F.col("route"))
    .limit(30)
)

most_frequent_routes_df = most_frequent_routes_df.collect()

for row in most_frequent_routes_df:
    print(f'{row["route"]}\t{row["cnt"]}')

main	8184
main-archive	1113
main-rabota	1047
main-internet	897
main-bonus	870
main-news	769
main-tariffs	677
main-online	587
main-vklad	518
main-rabota-archive	170
main-archive-rabota	167
main-bonus-archive	143
main-rabota-bonus	139
main-bonus-rabota	135
main-news-rabota	135
main-archive-internet	132
main-internet-rabota	129
main-rabota-news	129
main-archive-news	126
main-rabota-internet	124
main-internet-archive	123
main-archive-bonus	117
main-internet-bonus	115
main-tariffs-internet	114
main-news-archive	113
main-news-internet	109
main-archive-tariffs	104
main-internet-news	103
main-tariffs-archive	103
main-rabota-main	94


* **Let's save top 10 to result.json**

In [11]:
import json

routes_dict = {row['route']: row['cnt'] for row in most_frequent_routes_spark_sql[:10]}

with open("result.json", "w") as f:
    json.dump(routes_dict, f, indent=4)

print("The top 10 most frequent routes have been saved to result.json")

The top 10 most frequent routes have been saved to result.json


* **And compare our results:**

In [18]:
combined_routes = zip(
    most_frequent_routes_spark_sql, 
    most_frequent_routes_rdd, 
    most_frequent_routes_df
)

print("The 30 most frequent user routes:")
print(f"{'Route (Spark SQL)':<25} {'Count':<10} | {'Route (RDD)':<25} {'Count':<10} | {'Route (DataFrame)':<25} {'Count':<10}")

for row_sql, row_rdd, row_df in combined_routes:
    print(f"{row_sql['route']:<25} {row_sql['cnt']:<10} | {row_rdd[0]:<25} {row_rdd[1]:<10} | {row_df['route']:<25} {row_df['cnt']:<10}") 

The 30 most frequent user routes:
Route (Spark SQL)         Count      | Route (RDD)               Count      | Route (DataFrame)         Count     
main                      8184       | main                      8184       | main                      8184      
main-archive              1113       | main-archive              1113       | main-archive              1113      
main-rabota               1047       | main-rabota               1047       | main-rabota               1047      
main-internet             897        | main-internet             897        | main-internet             897       
main-bonus                870        | main-bonus                870        | main-bonus                870       
main-news                 769        | main-news                 769        | main-news                 769       
main-tariffs              677        | main-tariffs              677        | main-tariffs              677       
main-online               587        | main-on

## 4. Results

We observe that, for the most part, our results are consistent. However, there are occasional slight discrepancies between them.

- For example: main-rabota-news 130 | main-rabota-news 130 | main-internet-rabota 129

This variation can be attributed to the format of the timestamp field, which is represented as Unix timestamps in seconds. Since multiple pages can sometimes be visited at the same second, the order of events during the *order by* operation can become unpredictable, leading to different routes being generated in various instances.

If the timestamp were recorded in milliseconds, these discrepancies might not occur, as the more precise timing would better reflect the actual sequence of events.

Overall, despite these minor differences, our results appear to be reasonable and consistent.

In [13]:
sc.stop()